In [1]:
from sklearn.svm import SVC
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt

In [2]:
Train = pd.read_csv('.\\data\\train_v1.csv', index_col =0)

Label = Train['Loan_Status']
Train = Train.drop(columns='Loan_Status')
Test = pd.read_csv('.\\data\\test_v1.csv', index_col = 0)
Data = pd.concat([Train, Test])
print(Data.shape)
Data.describe()

(981, 16)


,Gender,Married,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
count,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000,981.000000
mean,0.209990,0.645260,0.777778,0.121305,5179.795107,1601.916330,142.511488,342.201835,0.849134,0.581040,0.163099,0.163099,0.092762,0.295617,0.355759,0.348624
std,0.407509,0.478679,0.415952,0.326648,5695.104533,2718.772806,76.347770,64.432886,0.358101,0.493641,0.369644,0.369644,0.290247,0.456552,0.478987,0.476778
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,0.000000,2875.000000,0.000000,101.000000,360.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,1.000000,0.000000,3800.000000,1110.000000,128.000000,360.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,1.000000,1.000000,0.000000,5516.000000,2365.000000,160.000000,360.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,81000.000000,41667.000000,700.000000,480.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# ApplicantIncome, CoapplicantIncome, LoanAmount, Loan_Amount_Term need to be nornalized

In [3]:
data = Data[['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term']].values
Data = Data.drop(columns=['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term'])
print(data)

[[5849.      0.    142.51  360.  ]
 [4583.   1508.    128.    360.  ]
 [3000.      0.     66.    360.  ]
 ...
 [3250.   1993.    126.    360.  ]
 [5000.   2393.    158.    360.  ]
 [9200.      0.     98.    180.  ]]


In [4]:
scaler = StandardScaler()
print(scaler.fit(data))
StandardScaler(copy=True, with_mean=True, with_std=True)
data = scaler.transform(data)

StandardScaler(copy=True, with_mean=True, with_std=True)


In [5]:
Data.index

Index(['LP001002', 'LP001003', 'LP001005', 'LP001006', 'LP001008', 'LP001011',
       'LP001013', 'LP001014', 'LP001018', 'LP001020',
       ...
       'LP002952', 'LP002954', 'LP002962', 'LP002965', 'LP002969', 'LP002971',
       'LP002975', 'LP002980', 'LP002986', 'LP002989'],
      dtype='object', name='Loan_ID', length=981)

In [6]:
Normalizaed = pd.DataFrame(data=data, columns=['ApplicantIncome', 'CoapplicantIncome', 'LoanAmount', 'Loan_Amount_Term'], index =Data.index ,copy=True)
Data = pd.merge(Data, Normalizaed, on='Loan_ID')
# Data['ApplicantIncome'].values = data[:,0]
# Data['CoapplicantIncome'].values = data[:,1]
# Data['LoanAmount'].values = data[:,2]
Train = Data[:Train.shape[0]]
Test = Data[Train.shape[0]:]
print(Train.shape)
print(Test.shape)
Data.head()

(614, 16)
(367, 16)


,Gender,Married,Education,Self_Employed,Credit_History,Dependents_0,Dependents_1,Dependents_2,Dependents_3+,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term
Loan_ID,,,,,,,,,,,,,,,,
LP001002,0,0,1,0,1.0,1,0,0,0,0,0,1,0.117565,-0.589506,-0.000020,0.276369
LP001003,0,1,1,0,1.0,0,1,0,0,1,0,0,-0.104844,-0.034561,-0.190168,0.276369
LP001005,0,1,1,1,1.0,1,0,0,0,0,0,1,-0.382944,-0.589506,-1.002656,0.276369
LP001006,0,1,0,0,1.0,1,0,0,0,0,0,1,-0.456202,0.278239,-0.295005,0.276369
LP001008,0,0,1,0,1.0,1,0,0,0,0,0,1,0.144093,-0.589506,-0.019808,0.276369


In [7]:
def binary2str(col):
    if col == 1:
        return 'Y'
    else:
        return 'N'

# PCA

In [8]:
pca = PCA(n_components=0.85)
pca.fit(Data.values)
#plt.plot(pca.explained_variance_ratio_)
Reduced_data = pca.transform(Data.values)
Reduced_data.shape

(981, 8)

In [9]:
Train = Data[:Train.shape[0]]
Test = Data[Train.shape[0]:]
print(Train.shape)
print(Test.shape)

(614, 16)
(367, 16)


# SVM

In [18]:
GAMMA = 10
C = 1

clf = SVC(gamma=GAMMA, C=C, class_weight = 'balanced')
clf.fit(Train.values, Label.values)
Out = clf.predict(Test.values)
print(Out)
print(len(Out))

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1
 1 0 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1]
367


In [19]:
Submit = pd.DataFrame({'Loan_ID': Test.index, 'Loan_Status': Out})
Submit['Loan_Status'] = Submit['Loan_Status'].apply(binary2str)#,axis = 1)
Submit.head()
Submit.to_csv('.\\data\\Submit_v1_SVM_C_'+str(C) + "_Gamma_" + str(GAMMA) +'_b8PCA.csv', index=False)

# Dicision tree

In [85]:
MAX_DEPTH = 16
clf = DecisionTreeClassifier(max_depth = MAX_DEPTH , class_weight = 'balanced' )
clf = clf.fit(Train.values, Label.values)
Out = clf.predict(Test.values)
print(Out)
print(len(Out))

[1 1 1 1 0 0 1 0 1 1 0 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 0 0 1 1 0 0 1 0 1 0 1 0
 0 0 1 1 1 0 0 1 0 1 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 0 0 0 0 0 1 1 0 0 0 0 1 0 1 1 0 1 1 0 1 1 0 1 1 0 1 0 1 1 1 1 0
 1 1 0 1 1 0 1 0 1 0 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 0 1 1 1 1 0 1 1 0 0 0 1 1 1 1 0 0 1 1 1 0
 0 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 0 1 1 0 1 0 1 1 0 1 1 1
 0 1 1 1 1 0 1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1
 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 0 1 1 0 0 1 0 1 1 1 0 0 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 1 0 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1]
367


In [ ]:
Submit = pd.DataFrame({'Loan_ID': Test.index, 'Loan_Status': Out})
Submit['Loan_Status'] = Submit['Loan_Status'].apply(binary2str)#,axis = 1)
Submit.head()
Submit.to_csv('.\\data\\Submit_v1_Dtree_max_depth_'+str(MAX_DEPTH)+'_b.csv', index=False)

# Random forest

In [89]:
N_estimator = 100
MAX_DEPTH = 2
Random_state = 0
clf = RandomForestClassifier(n_estimators=N_estimator, max_depth=MAX_DEPTH,random_state=Random_state, class_weight = 'balanced')
clf.fit(Train.values, Label.values)
Out = clf.predict(Test.values)
print(Out)
print(len(Out))

[1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 1 1 1 1 1 0 1 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
367


# MLP

In [32]:
HLS = 5
Hidden = -1
Alpha = 1e-4
BS = 256
clf = MLPClassifier(alpha = Alpha, hidden_layer_sizes=(HLS,), random_state=1, learning_rate = 'invscaling', max_iter = 1000, batch_size = BS)
clf.fit(Train.values, Label.values)
Out = clf.predict(Test.values)
print(Out)
print(len(Out))

[1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 0 1 1 1 1 0 1 1 0 0 1 0 1 1 1 1
 1 1 1 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 1 1
 1 1 1 1 1 1 0 0 0 1 1 1 0 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 0
 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 0 0 1 0 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1
 1 1 0 1 1 1 1 0 1 1 1 1 1 0 0 1 1 0 1 0 1 0 1 0 1 1 1 1 0 1 1 1 1 0 1 1 1
 0 1 1 1 1 1 1 0 0 0 1 1 1 1 0 0 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1
 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 0 1 1 1 1 1 1 0 1 1 1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1]
367


In [33]:
Submit = pd.DataFrame({'Loan_ID': Test.index, 'Loan_Status': Out})
Submit['Loan_Status'] = Submit['Loan_Status'].apply(binary2str)#,axis = 1)
Submit.head()
Submit.to_csv('.\\data\\Submit_v1_MLP_hidden_layer_sizes_'+str(HLS)+','+str(Hidden) +'_L2_'+str(Alpha) +"_BS_"+ str(BS) +'_8pca.csv', index=False)